In [2]:
import sys
from pathlib import Path
import matplotlib.pyplot as plt

# add directory with ribasim-repository
sys.path.append("..\\..\\ribasim\\python\\ribasim")
import ribasim

# add directory with ribasim_lumping-repository
sys.path.append("..")
import ribasim_lumping_nl

import pandas as pd
import geopandas as gpd
import numpy as np
import pyarrow as pa

from numba.core.errors import NumbaDeprecationWarning
import warnings

warnings.simplefilter("ignore", category=NumbaDeprecationWarning)
warnings.simplefilter("ignore", category=UserWarning)
warnings.simplefilter("ignore", category=FutureWarning)
pd.options.mode.chained_assignment = None

In [3]:
%load_ext autoreload
%autoreload 2

In [4]:
start_time = pd.Timestamp.now()

#### BASIS OPZETTEN

In [5]:
# define network name, base dir
network_name = "tki_hooge_raam"
base_dir = Path("..\\..\\ribasim_lumping_data\\")
# directory D-HYDRO basis calculations
dhydro_basis_dir = Path(base_dir, network_name, "2_dhydro\\basis\\")
# directory D-HYDRO results calculations
dhydro_results_dir = Path(base_dir, network_name, "2_dhydro\\results\\")
# directory results
results_dir = Path(base_dir, network_name, "3_ribasim\\")
# batch file used to calculate volume D-Hydro module per GridPoint
dhydro_volume_tool_bat_file = Path(r"c:\Program Files\Deltares\D-HYDRO Suite 2023.03 1D2D\plugins\DeltaShell.Dimr\kernels\x64\dflowfm\scripts\run_dfm_volume_tool.bat")
dhydro_volume_tool_increment = 0.1
# link to ribasim_executable
path_ribasim_executable = "..\\..\\..\\ribasim_cli\\bin\\ribasim.exe"

In [8]:
# Create networkanalysis
network = ribasim_lumping_nl.create_ribasim_lumping_network(
    base_dir=base_dir,
    name=network_name,
    dhydro_basis_dir=dhydro_basis_dir,
    dhydro_results_dir=dhydro_results_dir,
    hydamo_basis_dir = '' ,
    results_dir=results_dir,
    path_ribasim_executable=path_ribasim_executable,
    crs=28992
)
# network.export_to_geopackage()

#### INLEZEN NETWERK EN SIMULATIE DATA

In [9]:
# Load areas (discharge units: afwaterende eenheden)
areas_dir               = Path(base_dir, network_name, "1_areas")
areas_file_name         = "afwateringseenheden2023_v1_HoogeRaam.shp"
areas_id_column         = "CODE"
network.read_areas(
    areas_file_path=Path(areas_dir, areas_file_name), 
    areas_id_column=areas_id_column
)

Custom TB Handler failed, unregistering


Unexpected exception formatting exception. Falling back to standard exception


Traceback (most recent call last):
  File "fiona\ogrext.pyx", line 136, in fiona.ogrext.gdal_open_vector
  File "fiona\_err.pyx", line 291, in fiona._err.exc_wrap_pointer
fiona._err.CPLE_OpenFailedError: ..\..\ribasim_lumping_data\tki_hooge_raam\1_areas\afwateringseenheden2023_v1_HoogeRaam.shp: No such file or directory

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "c:\ProgramData\Anaconda3\envs\ribasim\Lib\site-packages\IPython\core\interactiveshell.py", line 3460, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "C:\Users\NLTAND\AppData\Local\Temp\ipykernel_3128\543981536.py", line 5, in <module>
    network.read_areas(
  File "c:\Users\NLTAND\OneDrive - Sweco AB\Algemeen-Tessa\Projecten\TKI oppervlaktewatermodule NHI\ribasim_lumping_NL\notebooks\..\ribasim_lumping_nl\ribasim_lumping.py", line 113, in read_areas
    def read_areas(self, areas_file_path: Path, areas_id_column: str):
             

In [10]:
# Read network data and extract all objects (weirs/pumps/laterals/confluences/bifurcations)
network.add_basis_network(
    source_type='dhydro',
    set_name="winter",
    model_dir=Path(dhydro_basis_dir, "winter"),
    simulation_name="0_01",
    dhydro_volume_tool_bat_file=dhydro_volume_tool_bat_file, 
    dhydro_volume_tool_force=True,
    dhydro_volume_tool_increment=dhydro_volume_tool_increment
);
# network.add_basis_network(
#     source_type='dhydro',
#     set_name='winter2', 
#     model_dir=Path(dhydro_basis_dir, "winter"),
#     simulation_name="HoogeRaamRibasim.dsproj_data",
#     dhydro_volume_tool_bat_file=dhydro_volume_tool_bat_file, 
#     dhydro_volume_tool_force=True,
#     dhydro_volume_tool_increment=0.1
# );

PermissionError: [Errno 13] Permission denied: '.'

In [ ]:
# Select simulation sets and extract all data using xugrid/ugrid
network.add_simulation_set(
    source_type='dhydro',
    set_name="winter",
    model_dir=Path(dhydro_results_dir, "winter"),
    simulation_names=["0_01", "0_02", "0_05", "0_075", "0_1", "0_2", "0_5", "0_75", "1_0", "1_5", "2_0"],
    simulation_ts=[-1],
);
# network.add_simulation_set(
#     source_type='dhydro',
#     set_name="zomer",
#     model_dir=Path(dhydro_results_dir, "winter"),
#     simulation_names=["0_01", "0_02", "0_05", "0_075", "0_1", "0_2", "0_5", "0_75", "1_0", "1_5", "2_0"],
#     simulation_ts=[-1],
# );

#### OPZETTEN SIMULATIE

In [ ]:
simulation_code = "test_14f_gebieden"
set_name = 'winter'
starttime = "2020-01-01 00:00:00"
endtime = "2020-02-01 00:00:00"

Define locations where the network should be split into Ribasim basins:

In [ ]:
# black list: "105BGA"
network.add_split_nodes(
    weirs=False,
    pumps=False,
    uniweirs=False,
    edges=False,
    structures_ids_to_include=[
        "105PAS", "105HOE", "105PVS", "105BZS",                         # 5 gebieden
        "105BRG", "105PCS", "105ONS", "105ACS", "105ADS",               # 10 gebieden
        # "105NBG", "105PVS", "105AGS", "105LBW", "105BGA",             # 14a gebieden
        # "105DHA", "105PVS", "105AGS", "105LBW", "105BGA",             # 14b gebieden
        # "105DHA", "105NBG", "105AGS", "105LBW", "105BGA",             # 14c gebieden
        # "105DHA", "105NBG", "105PVS", "105LBW", "105BGA",             # 14d gebieden
        # "105DHA", "105NBG", "105PVS", "105AGS", "105BGA",             # 14e gebieden
        "105DHA", "105NBG", "105PVS", "105AGS", "105LBW",             # 14f gebieden
        # "105DHA", "105NBG", "105PVS", "105AGS", "105LBW", "105GLS"      # 15 gebieden
        # "105DHA", "105NBG", "105PVS", "105AGS", "105LBW", "105BGA"      # 15 gebieden slow
        
        # '105MKP', '105PDS', '105AFT', 
        # '105GLS', '105ADH', '105WEV',
        # '105NBG', '105SBG', '105OMK', 
        # '105NXV', '105BZS', '105VLW',
        # '105PDT', '105OUS', '105OVS', 
        # '105OQS', '105OXS', '105LHW',
        # '105AGS', '105AHS', '105DHA', 
        # '105PET', '105PVS', '105HOE',
        # '105PGS', '105PES', '105BZV', 
        # '105ABS', '105ACS', '105PCL',
        # '105BRM', '105PBK', '105SAW', 
        # '105NXS', '105ONS', '105OMS',
        # '105BFA', '105MPD', '105LBW', 
        # '105PLW', '105PCS', '105ADB',
        # '105PBR', '105HHT', '105AKS', 
        # '105HOO', '105PAS', #'105BGA', 
        # '105RKW', '105ADK', '105PDR', 
        # '105AFS', '105AES', '105ADS',
        # '105BUS', '105ZAN', '105BRG',
    ],
    structures_ids_to_exclude=["105BBW"],
    edge_ids_to_include=[],                                  # 1 edge voor 15 gebieden
    edge_ids_to_exclude=[],
);

In [2]:
network.split_nodes.split_node_id.values

NameError: name 'network' is not defined

Specify translation split_nodes to ribasim-nodes per type or for specifiek split_node_id

In [ ]:
split_node_type_conversion = dict(
    weir="TabulatedRatingCurve",
    uniweir="TabulatedRatingCurve",
    pump="Pump",
    culvert="TabulatedRatingCurve",
    edge="ManningResistance",
    orifice="Outlet",
    boundary_connection="ManningResistance"
)
split_node_id_conversion = {}

Create basins (gdf) based on nodes, edges, split_node_ids and areas

In [ ]:
network.generate_ribasim_lumping_network(
    simulation_code=simulation_code,
    split_node_type_conversion=split_node_type_conversion,
    split_node_id_conversion=split_node_id_conversion,
);

Generate Ribasim model and export to geopackage

In [ ]:
ribasim_model = network.generate_ribasim_model_complete(
    set_name=set_name,
    starttime=starttime,
    endtime=endtime
)

In [ ]:
# network.plot_basin_waterlevels_for_basins(
#     set_name='winter', 
#     basins_nos=[6, 9, 10, 39, 19, 57, 58, 53, 20, 38]
# );

In [ ]:
f"Duration: {pd.Timestamp.now() - start_time}"

In [ ]:
basin_df, control_df, flow_df = network.read_ribasim_results(simulation_code=simulation_code)
basin_df.drop(columns='node_id').groupby('time').sum()['storage'].plot()

In [ ]:
flow_df[~flow_df['edge_id'].isna()].groupby('to_node_id').get_group(1)['flow'].plot()

In [ ]:
# q = uniweirs_q_gdf.loc['weirgen_discharge'].iloc[5]['winter']

In [ ]:
# import numpy as np
# import matplotlib.pyplot as plt
# from scipy.optimize import curve_fit

# for i, weir in uniweirs_q_gdf.loc['weirgen_discharge']['winter'].iterrows():
#     x_data = weir.index.values
#     y_data = weir.values

#     def exponential_function(x, a, b):
#         return a * np.exp(b * x)

#     params, covariance = curve_fit(exponential_function, x_data, y_data)
#     a, b = params

#     y_fit = exponential_function(x_data, a, b)

#     fig, ax = plt.subplots()
#     ax.scatter(x_data, y_data, label="Data")
#     ax.plot(x_data, y_fit, "-x", label="Exponential Fit", color='red')
#     fig.show()
#     # plt.legend()
#     # plt.xlabel('X')
#     # plt.ylabel('Y')
#     # plt.title('Exponential Regression')
#     # plt.show()

In [ ]:
# def exponential_function(x, a, b):
#     return a * np.exp(b * x)

# def apply_exponential_function(df_series):
#     x_data = df_series.index.values
#     y_data = df_series.values
#     params, covariance = curve_fit(exponential_function, x_data, y_data)
#     a, b = params
#     y_fit = exponential_function(x_data, a, b)
#     df_series.values = y_fit
#     return df_series

# uniweirs_q_gdf#.loc['winter']#.apply(lambda row: apply_exponential_function(row))